
## Mi intento Memo

## Paso 1 
Definimos nuestro una lista de palabras de prueba:

In [ ]:
words = ["Gato", "Escuela", "Perro", "Niño", "Casa"]


Creación de red neuraonal para la conversión de español a nahualt (https://huggingface.co/hackathon-pln-es/t5-small-spanish-nahuatl)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained('hackathon-pln-es/t5-small-spanish-nahuatl')
tokenizer = AutoTokenizer.from_pretrained('hackathon-pln-es/t5-small-spanish-nahuatl')
model.eval();

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]


Definimos una función para pasar una palabra de español a nahualt utilizando el modelo creado:

In [ ]:
def word_spanish_nahualt(word, model, tokenizer):
    input_ids = tokenizer('translate Spanish to Nahuatl: ' + word, return_tensors='pt').input_ids # tensores
    outputs = model.generate(input_ids) # tensores
    word_nahuatl = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0] # word in nahuatl
    return word_nahuatl



Le damos una lista de palabras:

In [ ]:
words_nahuatl = []
for word in words:
    words_nahuatl.append(word_spanish_nahualt(word, model, tokenizer))

In [ ]:
words_nahuatl

['tsontli', 'tlajkuiloa', 'chichi', 'pili', 'kali']

### Paso 2) Decir oraciones con esas palabras en español (Voz)
Por ejemplo: "Mi miztli tiene hambre"

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

Grabamos la oracion de prueba:

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

Creamos nuestro audio y con ayuda ffmpeg los convertimos en .wav

In [ ]:
with open('./grabaciones-webm/test.webm', 'wb') as f:
    f.write(recorder.audio.value)

!ffmpeg -i ./grabaciones-webm/test.webm -ac 1 -f wav ./grabaciones-wav/test.wav -y -hide_banner -loglevel panic

**Modelo voz español -> texto** (https://huggingface.co/patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm)

Utilizamos el modelo huggin face y probamos con nuestra oración:

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm")
audio_paths = ["./grabaciones-wav/test.wav"]

transcriptions_es = model.transcribe(audio_paths)[0]

04/28/2022 19:08:36 - INFO - huggingsound.speech_recognition.model - Loading model...
/root/venv/lib/python3.7/site-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
100%|██████████| 1/1 [00:03<00:00,  3.46s/it]


In [ ]:
transcriptions_es['transcription']

'hola cómo estás'

**Modelo voz nahual -> texto**
https://huggingface.co/tyoc213/wav2vec2-large-xlsr-nahuatl

Utilizamos el modelo huggin face y probamos con nuestra oración:

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("tyoc213/wav2vec2-large-xlsr-nahuatl")
audio_paths = ["./grabaciones-wav/audio.mp3"]

transcriptions_nah = model.transcribe(audio_paths)[0]

04/28/2022 19:18:15 - INFO - huggingsound.speech_recognition.model - Loading model...
/root/venv/lib/python3.7/site-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
04/28/2022 19:18:27 - WARNING - root - bos_token <s> not in provided tokens. It will be added to the list of tokens
04/28/2022 19:18:27 - WARNING - root - eos_token </s> not in provided tokens. It will be added to the list of tokens
100%|██████████| 1/1 [00:03<00:00,  3.78s/it]


In [ ]:
transcriptions_nah['transcription']

"nas yoxofio no y n' ei"

**Observacion: Todavia no hay un traductor de nahual al español**

## Referecia
- https://huggingface.co/hackathon-pln-es/t5-small-spanish-nahuatl (Existen diferente versiones de nahual para escribirlo)
- https://huggingface.co/patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm
- https://huggingface.co/tyoc213/wav2vec2-large-xlsr-nahuatl
- https://huggingface.co/spaces?sort=modified&search=spee (Ejemplos)
- https://colab.research.google.com/gist/omarespejel/af89fe202db90c21320c1313b6fe2e2f#scrollTo=ayOhsxluT8Xj

**Audios en jupyter notebooks**
- https://medium.com/@harrycblum/record-audio-in-a-jupyter-notebook-da08a88278bb
- https://stackoverflow.com/questions/52442361/record-audio-in-jupyter-notebook
- https://ipywebrtc.readthedocs.io/en/latest/api.html#module-ipywebrtc.webrtc
- https://ipywebrtc.readthedocs.io/en/latest/index.html
- https://stackoverflow.com/questions/156767/whats-the-difference-between-an-argument-and-a-parameter

---

### Voice code

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
with open('./grabaciones-webm/testmemo.webm', 'wb') as f:
    f.write(recorder.audio.value)

!ffmpeg -i ./grabaciones-webm/testmemo.webm -ac 1 -f wav ./grabaciones-wav/testmemo.wav -y -hide_banner -loglevel panic

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm")

04/30/2022 16:57:29 - INFO - huggingsound.speech_recognition.model - Loading model...
/root/venv/lib/python3.7/site-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
audio_paths = ["./grabaciones-wav/test.wav"]

# Obtenemos la frase del usuario
transcriptions_es = model.transcribe(audio_paths)[0]

100%|██████████| 1/1 [00:01<00:00,  1.90s/it]


In [ ]:
def sim_jac(s1, s2):

    bigrams_s1 = []
    bigrams_s2 = []
    
    for i in range(len(s1) - 1):
        bigrams_s1.append(s1[i:i+2])
    
    for i in range(len(s2) - 1):
        bigrams_s2.append(s2[i:i+2])

    c_common = 0

    for i in bigrams_s1:
        if bigrams_s2.count(i) > 0:
            c_common += 1

    return c_common / ((len(s1) - 1) + (len(s2) - 1) - c_common)

In [ ]:
def definir_funcion():
    code = """
    def nombre_function():
        pass
    """
    return code

In [ ]:
frase_usuario = transcriptions_es['transcription']

# Ver similitud
sim_jac('definir función', frase_usuario)

1.0

In [ ]:
frase_usuario

'definir función'

In [ ]:
def funcion(x, y):
    for i in range(50):
        print(f'Paso {i} listo')
    return x + y

In [ ]:
a = 3
b = 7
# voz llamar funcion: (nombre de la func)
# definir argumentos
funcion(a, b)

Paso 0 listo
Paso 1 listo
Paso 2 listo
Paso 3 listo
Paso 4 listo
Paso 5 listo
Paso 6 listo
Paso 7 listo
Paso 8 listo
Paso 9 listo
Paso 10 listo
Paso 11 listo
Paso 12 listo
Paso 13 listo
Paso 14 listo
Paso 15 listo
Paso 16 listo
Paso 17 listo
Paso 18 listo
Paso 19 listo
Paso 20 listo
Paso 21 listo
Paso 22 listo
Paso 23 listo
Paso 24 listo
Paso 25 listo
Paso 26 listo
Paso 27 listo
Paso 28 listo
Paso 29 listo
Paso 30 listo
Paso 31 listo
Paso 32 listo
Paso 33 listo
Paso 34 listo
Paso 35 listo
Paso 36 listo
Paso 37 listo
Paso 38 listo
Paso 39 listo
Paso 40 listo
Paso 41 listo
Paso 42 listo
Paso 43 listo
Paso 44 listo
Paso 45 listo
Paso 46 listo
Paso 47 listo
Paso 48 listo
Paso 49 listo


10

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=17740a58-e960-4de3-bb66-1575286930f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>